## Prerequisites - Environment Setup

Before starting this workshop, set up a .NET environment and add your Azure credentials.

### Step 1: Install .NET SDK
Ensure you have .NET 9.0 or later installed:
```bash
dotnet --version
```

### Step 2: Create `.env`
Copy the shared template, then update the values with your Azure resource details:
```bash
# Run from Backend/dotnet/agentframework
copy ..\env.template .env          # Windows (PowerShell)
cp ../env.template .env            # macOS/Linux
```

Edit `.env` and provide at least:
```bash
AZURE_OPENAI_ENDPOINT=https://<resource>.openai.azure.com
AZURE_OPENAI_API_KEY=<key>
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o
AZURE_OPENAI_API_VERSION=2024-10-21

# Optional but required for Azure AI Foundry agents
PROJECT_ENDPOINT=https://<resource>.services.ai.azure.com/api/projects/<project>
PEOPLE_AGENT_ID=asst-people-agent
KNOWLEDGE_AGENT_ID=asst-knowledge-agent
MANAGED_IDENTITY_CLIENT_ID=<client-id-if-using-UMI>
```

**Important:** `.env` contains secrets—keep it out of source control (`.gitignore` already excludes it).

### Step 3: Select .NET Kernel in VS Code
1. Click the **kernel selector** (top-right of this notebook)
2. Choose **"Select Another Kernel..."**
3. Pick **".NET Interactive"**
4. Select the C# kernel

---

# .NET Agent Framework Workshop

This notebook demonstrates modern AI agent development using Microsoft Agent Framework in .NET with Azure OpenAI services.

## Workshop Focus Areas
1. **Generic Agent** - Build basic conversational agents using Agent Framework
2. **Azure AI Foundry Agents** - Demonstrate specialized agents like PeopleLookupAgent and KnowledgeFinderAgent
3. **Group Chat System** - Multi-agent collaboration with built-in orchestration

## Key Technologies
- **Microsoft Agent Framework** - Microsoft's next-generation AI orchestration framework for .NET
- **Azure OpenAI** - GPT-4 models for intelligent responses
- **Azure AI Foundry** - Cloud-based specialized AI agents
- **Agent Coordination** - Multi-agent systems for complex tasks

Let's build sophisticated AI agent systems using .NET and the Microsoft Agent Framework!

In [ ]:
// Environment setup and configuration loading
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: NetEscapades.Configuration.Yaml, 3.1.0"

using System.IO;
using Microsoft.Extensions.Configuration;
using NetEscapades.Configuration.Yaml;

// Check required environment variables
string[] requiredEnvVars = {
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "AZURE_OPENAI_DEPLOYMENT_NAME",
    "PROJECT_ENDPOINT",
    "PEOPLE_AGENT_ID",
    "KNOWLEDGE_AGENT_ID"
};

Console.WriteLine("🔍 Checking environment variables...");
foreach (var envVar in requiredEnvVars)
{
    var value = Environment.GetEnvironmentVariable(envVar);
    if (string.IsNullOrEmpty(value))
    {
        Console.WriteLine($"❌ Missing: {envVar}");
    }
    else
    {
        Console.WriteLine($"✅ Found: {envVar}");
    }
}

// Load configuration from config.yml using .NET Configuration
var configuration = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddYamlFile("config.yml", optional: true, reloadOnChange: true)
    .Build();

Console.WriteLine("\n📄 Configuration system loaded");
Console.WriteLine("✅ Ready to create agents with config-driven instructions!");

// Helper method to get agent instructions from config
static string GetAgentInstructions(IConfiguration config, string agentName, string fallbackInstructions)
{
    var instructions = config[$"agents:{agentName}:instructions"];
    return !string.IsNullOrEmpty(instructions) ? instructions : fallbackInstructions;
}

### Step 1: Install Required Packages

In [ ]:
// Install and load the DotEnv package for environment variables
#r "nuget: DotNetEnv, 3.1.1"

using DotNetEnv;

// Load environment variables from .env file
Env.Load();

Console.WriteLine("✓ Environment variables loaded from .env file");
Console.WriteLine("✓ All modules imported successfully!");
Console.WriteLine("Ready to configure agents and start building!");

In [ ]:
// Verify environment configuration
var requiredVars = new[] { "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_DEPLOYMENT_NAME" };
var optionalFoundryVars = new[] { "AZURE_OPENAI_API_VERSION", "PROJECT_ENDPOINT", "PEOPLE_AGENT_ID", "KNOWLEDGE_AGENT_ID", "MANAGED_IDENTITY_CLIENT_ID" };

Console.WriteLine("Checking required environment variables ...");
var missingVars = new List<string>();

foreach (var envVar in requiredVars)
{
    var value = Environment.GetEnvironmentVariable(envVar);
    if (!string.IsNullOrEmpty(value))
    {
        Console.WriteLine($"✓ {envVar}: {value.Substring(0, Math.Min(24, value.Length))}...");
    }
    else
    {
        missingVars.Add(envVar);
        Console.WriteLine($"❌ {envVar}: Not set");
    }
}

if (missingVars.Any())
{
    Console.WriteLine($"\n❌ Missing required variables: {string.Join(", ", missingVars)}");
    Console.WriteLine("Populate them in .env before invoking Azure OpenAI.");
}
else
{
    Console.WriteLine("\n✓ All required Azure OpenAI variables are configured!");
}

Console.WriteLine("\nOptional Azure AI Foundry variables:");
foreach (var envVar in optionalFoundryVars)
{
    var value = Environment.GetEnvironmentVariable(envVar);
    var status = !string.IsNullOrEmpty(value) ? "✓" : "•";
    var hint = !string.IsNullOrEmpty(value) ? $"{value.Substring(0, Math.Min(24, value.Length))}..." : "(not set)";
    Console.WriteLine($"{status} {envVar}: {hint}");
}

Console.WriteLine("\nFoundry variables enable enterprise agents. It's safe to continue without them if you're focusing on Azure OpenAI only.");

In [ ]:
// Install required packages with latest versions for Agent Framework
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251002.1"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.AI.Projects, 1.0.0-beta.11"
#r "nuget: Azure.Identity, 1.16.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.8"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.8"
#r "nuget: Microsoft.Extensions.Logging.Console, 9.0.8"

// Create a Generic Agent using Microsoft Agent Framework with config.yml instructions
using Microsoft.Agents.AI;
using Azure.AI.OpenAI;
using OpenAI;
using OpenAI.Chat;
using Azure.Identity;
using System.ComponentModel;

// Agent configuration class
public class AgentConfig
{
    public string Name { get; set; }
    public string AgentType { get; set; }
    public Dictionary<string, string> FrameworkConfig { get; set; }
    public string Instructions { get; set; }
}

// Generic Agent Response class
public class AgentResponse
{
    public string Content { get; set; }
    public string AgentName { get; set; }
}

// Generic Agent Framework Agent class
public class AgentFrameworkGenericAgent
{
    public string Name { get; private set; }
    public AgentConfig Config { get; private set; }
    private ChatClient _chatClient;

    public AgentFrameworkGenericAgent(AgentConfig config)
    {
        Config = config;
        Name = config.Name;
    }

    public async Task InitializeAsync()
    {
        // Configure Azure OpenAI using the new Agent Framework patterns
        var endpoint = Config.FrameworkConfig["endpoint"];
        var apiKey = Config.FrameworkConfig["api_key"];
        var deploymentName = Config.FrameworkConfig["deployment_name"];

        var azureClient = new AzureOpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(apiKey));
        _chatClient = azureClient.GetChatClient(deploymentName);
    }

    public async Task<AgentResponse> ProcessMessageAsync(string message)
    {
        var messages = new List<ChatMessage>
        {
            new SystemChatMessage(Config.Instructions),
            new UserChatMessage(message)
        };

        var response = await _chatClient.CompleteChatAsync(messages);
        
        return new AgentResponse
        {
            Content = response.Value.Content[0].Text ?? "I apologize, but I couldn't generate a response.",
            AgentName = Name
        };
    }
}

// Get instructions from config.yml with fallback
var genericInstructions = GetAgentInstructions(configuration, "generic_agent", 
    @"You are a helpful AI assistant that can answer questions and have conversations.
        
        Your capabilities include:
        - Answering general questions
        - Helping with problem-solving
        - Providing explanations and information
        - Having natural conversations
        
        Always be helpful, accurate, and professional in your responses.");

// Create and initialize a generic Agent Framework agent using config.yml instructions
var config = new AgentConfig
{
    Name = "GenericAssistant",
    AgentType = "GENERIC",
    FrameworkConfig = new Dictionary<string, string>
    {
        ["endpoint"] = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT"),
        ["api_key"] = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY"),
        ["deployment_name"] = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME")
    },
    Instructions = genericInstructions  // Using config.yml instructions!
};

var genericAgent = new AgentFrameworkGenericAgent(config);
await genericAgent.InitializeAsync();

Console.WriteLine("✓ Generic Agent Framework Agent created successfully!");
Console.WriteLine($"Agent Name: {genericAgent.Name}");
Console.WriteLine($"Agent Type: {genericAgent.Config.AgentType}");
Console.WriteLine("📄 Instructions loaded from config.yml");
Console.WriteLine("Ready to test the agent!");

### Test the Generic Agent

In [ ]:
// Test the Generic Agent
var userMessage = "Hello! What is artificial intelligence and how does it work?";

Console.WriteLine($"User: {userMessage}");
Console.WriteLine("Sending message to agent...");

try
{
    // Get response from the agent
    var response = await genericAgent.ProcessMessageAsync(userMessage);
    
    Console.WriteLine($"\nAgent ({genericAgent.Name}): {response.Content}");
    Console.WriteLine($"\nResponse received successfully!");
    Console.WriteLine("✓ Generic agent test completed!");
    Console.WriteLine("You can modify the userMessage above to test different questions.");
}
catch (Exception e)
{
    Console.WriteLine($"❌ Error testing agent: {e.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration.");
}

In [ ]:
// Create Azure AI Foundry Specialized Agents using config.yml instructions
// Azure Foundry Agent class using Agent Framework
public class AgentFrameworkAzureFoundryAgent
{
    public string Name { get; private set; }
    public AgentConfig Config { get; private set; }
    private ChatClient _chatClient;

    public AgentFrameworkAzureFoundryAgent(AgentConfig config)
    {
        Config = config;
        Name = config.Name;
    }

    public async Task InitializeAsync()
    {
        // Configure Azure OpenAI (for demo purposes, using OpenAI instead of Azure AI Foundry)
        // In production, this would connect to Azure AI Foundry agents using the Agent Framework
        var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
        var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
        var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME");

        var azureClient = new AzureOpenAIClient(new Uri(endpoint), new Azure.AzureKeyCredential(apiKey));
        _chatClient = azureClient.GetChatClient(deploymentName);
    }

    public async Task<AgentResponse> ProcessMessageAsync(string message)
    {
        var messages = new List<ChatMessage>
        {
            new SystemChatMessage(Config.Instructions),
            new UserChatMessage(message)
        };

        var response = await _chatClient.CompleteChatAsync(messages);
        
        return new AgentResponse
        {
            Content = response.Value.Content[0].Text ?? "I apologize, but I couldn't generate a response.",
            AgentName = Name
        };
    }
}

// Get instructions from config.yml with fallback
var peopleInstructions = GetAgentInstructions(configuration, "people_lookup",
    @"You are a People Lookup Agent, an expert at finding and providing information about people, contacts, and team members.

Your expertise includes:
• Directory Searches: Finding people by name, role, department, or skills
• Team Coordination: Understanding organizational structure and relationships  
• Contact Discovery: Providing appropriate contact information
• Role Identification: Matching people to specific tasks or expertise needs

Guidelines for responses:
- Be helpful and accurate when providing people information
- If you don't know a specific person, explain what information would be helpful to find them
- Suggest ways to locate people or get contact information
- Always try to be helpful for any people-related query");

var knowledgeInstructions = GetAgentInstructions(configuration, "knowledge_finder",
    @"You are a Knowledge Finder Agent, a specialist at searching, retrieving, and organizing information from various knowledge sources.

Your expertise includes:
• Document Search: Finding relevant documents, wikis, and knowledge bases
• Research Assistance: Discovering and synthesizing information from multiple sources
• Information Organization: Structuring and summarizing complex knowledge
• Context Provision: Connecting related information and concepts

Guidelines for responses:
- Focus on finding the most relevant and accurate information
- Provide clear summaries with key points highlighted
- Help users understand complex information through clear explanations
- Structure information in a logical, easy-to-digest format");

// People Lookup Agent using config.yml instructions
var peopleConfig = new AgentConfig
{
    Name = "PeopleLookupAgent",
    AgentType = "PEOPLE_LOOKUP",
    FrameworkConfig = new Dictionary<string, string>
    {
        ["agent_id"] = Environment.GetEnvironmentVariable("PEOPLE_AGENT_ID"),
        ["project_endpoint"] = Environment.GetEnvironmentVariable("PROJECT_ENDPOINT")
    },
    Instructions = peopleInstructions  // Using config.yml instructions!
};

// Knowledge Finder Agent using config.yml instructions
var knowledgeConfig = new AgentConfig
{
    Name = "KnowledgeFinderAgent",
    AgentType = "KNOWLEDGE_FINDER",
    FrameworkConfig = new Dictionary<string, string>
    {
        ["agent_id"] = Environment.GetEnvironmentVariable("KNOWLEDGE_AGENT_ID"),
        ["project_endpoint"] = Environment.GetEnvironmentVariable("PROJECT_ENDPOINT")
    },
    Instructions = knowledgeInstructions  // Using config.yml instructions!
};

// Create and initialize the agents
var peopleAgent = new AgentFrameworkAzureFoundryAgent(peopleConfig);
var knowledgeAgent = new AgentFrameworkAzureFoundryAgent(knowledgeConfig);

await peopleAgent.InitializeAsync();
await knowledgeAgent.InitializeAsync();

Console.WriteLine("✓ Azure AI Foundry agents created successfully using Agent Framework!");
Console.WriteLine($"1. {peopleAgent.Name} - Specializes in people and contact information");
Console.WriteLine($"2. {knowledgeAgent.Name} - Expert at finding and organizing knowledge");
Console.WriteLine("📄 All agent instructions loaded from config.yml");
Console.WriteLine("These agents demonstrate specialized capabilities powered by Microsoft Agent Framework!");

In [ ]:
// Test Azure AI Foundry Agents
Console.WriteLine("Testing Azure AI Foundry specialized agents...");
Console.WriteLine();

// Test cases for each agent
var testCases = new[]
{
    new { Agent = peopleAgent, Question = "I need to find someone who can help with C# development. Who should I contact?" },
    new { Agent = knowledgeAgent, Question = "What are the best practices for implementing microservices architecture?" }
};

for (int i = 0; i < testCases.Length; i++)
{
    var testCase = testCases[i];
    
    try
    {
        Console.WriteLine($"Test {i + 1}: {testCase.Agent.Name}");
        Console.WriteLine($"Question: {testCase.Question}");
        Console.WriteLine(new string('-', 50));
        
        // Get response from the agent
        var response = await testCase.Agent.ProcessMessageAsync(testCase.Question);
        Console.WriteLine($"Response: {response.Content}");
        
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
    catch (Exception e)
    {
        Console.WriteLine($"❌ Error testing {testCase.Agent.Name}: {e.Message}");
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
}

Console.WriteLine("✓ Specialized agent testing completed!");
Console.WriteLine("Each agent demonstrated expertise in their specialized domain using Agent Framework.");

### Create Group Chat with All Three Agents

In [ ]:
// Create Group Chat with Multiple Agents using Agent Framework
// Simple agent registry for managing agents
public class SimpleAgentRegistry
{
    private readonly Dictionary<string, object> _agents = new();

    public void RegisterAgent(string name, object agent)
    {
        _agents[name] = agent;
        Console.WriteLine($"Registered agent: {name}");
    }

    public T GetAgent<T>(string name) where T : class
    {
        return _agents.TryGetValue(name, out var agent) ? agent as T : null;
    }

    public List<string> GetAllAgentNames()
    {
        return _agents.Keys.ToList();
    }
}

// Simple group chat router using Agent Framework patterns
public class GroupChatRouter
{
    private readonly SimpleAgentRegistry _agentRegistry;
    private readonly Dictionary<string, string> _agentNameMapping;

    public GroupChatRouter(SimpleAgentRegistry agentRegistry)
    {
        _agentRegistry = agentRegistry;
        _agentNameMapping = new Dictionary<string, string>
        {
            ["generic_agent"] = "GenericAssistant",
            ["people_lookup"] = "PeopleLookupAgent",
            ["knowledge_finder"] = "KnowledgeFinderAgent",
            ["generic"] = "GenericAssistant"
        };
    }

    public async Task<AgentResponse> RouteMessageAsync(string message, string sessionId, string userId)
    {
        var availableAgents = _agentRegistry.GetAllAgentNames();
        
        Console.WriteLine($"Available agents for routing: {string.Join(", ", availableAgents)}");
        
        if (!availableAgents.Any())
        {
            throw new Exception("No agents available for routing");
        }

        // Simple routing logic: route based on keywords
        string selectedAgentName;
        if (message.ToLower().Contains("people") || message.ToLower().Contains("person") || message.ToLower().Contains("who"))
        {
            selectedAgentName = "PeopleLookupAgent";
        }
        else if (message.ToLower().Contains("knowledge") || message.ToLower().Contains("information") || message.ToLower().Contains("best practices"))
        {
            selectedAgentName = "KnowledgeFinderAgent";
        }
        else
        {
            selectedAgentName = "GenericAssistant";
        }

        Console.WriteLine($"Router selected: {selectedAgentName}");

        // Get the selected agent and process the message
        if (selectedAgentName == "GenericAssistant")
        {
            var agent = _agentRegistry.GetAgent<AgentFrameworkGenericAgent>(selectedAgentName);
            if (agent != null)
            {
                Console.WriteLine($"Processing message with agent: {agent.Name}");
                return await agent.ProcessMessageAsync(message);
            }
        }
        else
        {
            var agent = _agentRegistry.GetAgent<AgentFrameworkAzureFoundryAgent>(selectedAgentName);
            if (agent != null)
            {
                Console.WriteLine($"Processing message with agent: {agent.Name}");
                return await agent.ProcessMessageAsync(message);
            }
        }

        throw new Exception($"Agent {selectedAgentName} not found");
    }
}

// Create the group chat system using Agent Framework
Console.WriteLine("Creating Agent Framework Group Chat system...");

// Create session manager and simple agent registry
var agentRegistry = new SimpleAgentRegistry();

// Register all our agents directly
agentRegistry.RegisterAgent("GenericAssistant", genericAgent);
agentRegistry.RegisterAgent("PeopleLookupAgent", peopleAgent);
agentRegistry.RegisterAgent("KnowledgeFinderAgent", knowledgeAgent);

Console.WriteLine($"Registered agents: {string.Join(", ", agentRegistry.GetAllAgentNames())}");

// Create the group chat router
var groupChatRouter = new GroupChatRouter(agentRegistry);

Console.WriteLine("✓ Agent Framework Group Chat created successfully!");
Console.WriteLine($"Participants: {string.Join(", ", agentRegistry.GetAllAgentNames())}");
Console.WriteLine("Ready for multi-agent collaboration powered by Microsoft Agent Framework!");

In [ ]:
// Test Group Chat with Complex Question using Agent Framework
var complexQuestion = @"We're launching a new AI-powered project management tool. 
We need to understand:
1. Who should be involved in the development team?
2. What technical knowledge and best practices should we consider?
3. How can we ensure the project is successful?

Please provide a comprehensive response with insights from different perspectives.";

Console.WriteLine("Testing Agent Framework Group Chat with Complex Question:");
Console.WriteLine($"Question: {complexQuestion}");
Console.WriteLine(new string('=', 80));
Console.WriteLine();

try
{
    // Create a new session for this group chat
    var sessionId = "group_chat_test";
    
    // Send the question to the group chat router
    Console.WriteLine("Starting Agent Framework group chat collaboration...");
    var response = await groupChatRouter.RouteMessageAsync(complexQuestion, sessionId, "user_test");
    
    Console.WriteLine($"Group Chat Response:");
    Console.WriteLine($"{response.Content}");
    Console.WriteLine();
    Console.WriteLine("✓ Agent Framework Group Chat completed successfully!");
    Console.WriteLine("The agents worked together using Microsoft Agent Framework to provide a comprehensive response!");
}
catch (Exception e)
{
    Console.WriteLine($"❌ Error in group chat: {e.Message}");
    Console.WriteLine("Please check your Azure OpenAI and Azure AI Foundry configuration.");
}

### Try Your Own Group Chat Question

In [ ]:
// Try Your Own Group Chat Question with Agent Framework
Console.WriteLine("Custom Agent Framework Group Chat Test");
Console.WriteLine("Feel free to modify the question below to explore different topics!");
Console.WriteLine();

// MODIFY THIS QUESTION to test different scenarios:
var customQuestion = @"How can we improve remote team collaboration? 
Consider finding the right people, knowledge management, and general best practices.";

Console.WriteLine($"Your Question: {customQuestion}");
Console.WriteLine(new string('=', 60));

try
{
    // Create a session for the custom test
    var sessionId = "custom_group_chat";
    
    Console.WriteLine("Starting custom Agent Framework group chat...");
    var response = await groupChatRouter.RouteMessageAsync(customQuestion, sessionId, "custom_user");
    
    Console.WriteLine($"Group Response:");
    Console.WriteLine($"{response.Content}");
    Console.WriteLine();
    Console.WriteLine("✓ Custom Agent Framework group chat completed!");
    Console.WriteLine("Feel free to modify the customQuestion above to explore different topics!");
}
catch (Exception e)
{
    Console.WriteLine($"❌ Error in custom group chat: {e.Message}");
}

## Integration with .NET API Backend

This workshop connects with the actual .NET Agent Framework API backend. The agents you've created here follow the same patterns used in the production API.

### Key Integration Points:
1. **Agent Architecture** - Uses the same agent classes as the backend API powered by Microsoft Agent Framework
2. **Group Chat System** - Implements a simple router for multi-agent coordination  
3. **Azure Integration** - Same Azure OpenAI and Azure AI Foundry setup as the API
4. **Session Management** - Compatible session and agent registry patterns

### Production Features Available:
- **REST API Endpoints** - `/group-chat` for multi-agent conversations
- **Agent Registry** - Dynamic agent management and selection
- **Session Management** - Persistent conversation history
- **Error Handling** - Robust error handling and logging

### Try the API:
You can test the running API backend at `http://localhost:8000` (if running) with endpoints like:
- `GET /health` - Check system health
- `GET /agents` - List available agents  
- `POST /group-chat` - Start multi-agent conversations

The notebook agents you've created demonstrate the core capabilities that power the full API system using Microsoft Agent Framework!